# Setup

Ensure all dependencies are installed.


In [1]:
%pip install -U -r requirements.txt

  Cloning ssh://****@gitlab.utwente.nl/s2219735-bsc-research-project/mesh-to-points.git (to revision main) to /tmp/pip-install-evbkv0i_/m2p_2fc9d409ae594a8f8fc86cf8c979b9bb
  Running command git clone --filter=blob:none --quiet 'ssh://****@gitlab.utwente.nl/s2219735-bsc-research-project/mesh-to-points.git' /tmp/pip-install-evbkv0i_/m2p_2fc9d409ae594a8f8fc86cf8c979b9bb
  Resolved ssh://****@gitlab.utwente.nl/s2219735-bsc-research-project/mesh-to-points.git to commit 78899e62db635567c95b5e83565a036f4974d071
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


Check if we have GPU support, and if not, warn the user.


In [2]:
import torch
import warnings

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    warnings.warn("CUDA is not available. Running on CPU.")

CUDA is available!


Load our dataset...


In [3]:
import tempfile
import os
import zipfile
import shutil

import utils.file

from data_sources import DataSource, ModelNet40
from dataset import ModelNetDataset

data_source: DataSource = ModelNet40()
data_size = utils.file.get_download_size(data_source.url)

temp_dir = tempfile.gettempdir()
zip_path = os.path.join(temp_dir, data_source.name + ".zip")
dataset_path = os.path.join(temp_dir, data_source.name)

if os.path.exists(zip_path) and utils.file.checksum(zip_path) == data_source.checksum:
    print("Zip already downloaded.")
else:
    utils.file.download_file(data_source.url, zip_path)
    print("Zip downloaded.")

if os.path.exists(dataset_path):
    print("Conflicting data folder found. Deleting existing folder...")
    shutil.rmtree(dataset_path)
    print("Folder deleted.")

print("Extracting data...")
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(dataset_path)
    print("Data extracted.")

npoints = 2500

train_dataset = ModelNetDataset(root=dataset_path, npoints=npoints, split="train")

test_dataset = ModelNetDataset(
    root=dataset_path,
    split="test",
    npoints=npoints,
)

print(
    f"Training data: {len(train_dataset)} samples with {len(train_dataset.classes)} classes"
)
print(
    f"Test data size: {len(test_dataset)} samples with {len(test_dataset.classes)} classes"
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/tmp/ModelNet40.zip: 100%|███████████████████████████████████████████████████████████████████████| 1.90G/1.90G [01:04<00:00, 31.6MiB/s]


Zip downloaded.
Extracting data...
Data extracted.
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/chair/train/chair_0856.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/chair/train/chair_0092.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/chair/train/chair_0451.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/chair/train/chair_0452.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/sofa/train/sofa_0458.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/sofa/train/sofa_0409.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/sofa/train/sofa_0562.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/sofa/train/sofa_0640.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/sofa/train/sofa_0054.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/sofa/train/sofa_0325.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/sofa/train/sofa_0297.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/sofa/train/sofa_05

# Training

Train a new classifier or load a new pre-trained one:


In [4]:
from pointnet.train_classification import train_classification
from pointnet.model import PointNetCls

classifier_path = ""  # pointnet/classifier.pth

if classifier_path and os.path.exists(classifier_path):
    print(f"Loading existing model from {classifier_path}...")
    classifier = PointNetCls(k=len(train_dataset.classes))
    classifier.load_state_dict(torch.load(classifier_path, weights_only=True))
    print("Classifier loaded successfully.")
else:
    classifier = train_classification(
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        epochs=25,
        outf="./pointnet/cls",
    )

Random Seed:  7713


OutOfMemoryError: CUDA out of memory. Tried to allocate 314.00 MiB. GPU 0 has a total capacity of 14.57 GiB of which 12.75 MiB is free. Process 3079325 has 3.71 GiB memory in use. Process 3146416 has 2.06 GiB memory in use. Process 3171325 has 6.14 GiB memory in use. Process 3146413 has 1.40 GiB memory in use. Process 3245348 has 1.25 GiB memory in use. Of the allocated memory 880.33 MiB is allocated by PyTorch, and 267.67 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Run a simple classification to get a visualization of our classifier.


In [ ]:
import random
import torch
from utils.plot import plot_points

i = random.randint(0, len(test_dataset) - 1)

input_data = test_dataset[i][0].unsqueeze(0)
input_data = input_data.transpose(1, 2)

classifier.eval()

with torch.no_grad():
    output = classifier(input_data)


scores: torch.Tensor = output[0][0]
print("Scores:", scores.tolist())
print(
    f"Expected class: {test_dataset.classes[int(test_dataset[i][1])]} ({int(test_dataset[i][1])})",
)
print(
    f"Predicted class: {test_dataset.classes[scores.argmax().item()]} ({scores.argmax().item()})"
)

points = input_data[0].transpose(0, 1).numpy()
plot_points(points)

Train the segmenter:


In [ ]:
from pointnet.train_segmentation import train_segmentation

train_segmentation(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    epochs=25,
    class_choice=0,
    outf="./pointnet/seg",
)